Please download video from [here](https://youtu.be/EuLMrUFNRxQ) and place it in the same directory as this notebook.

In [ ]:
!git clone https://github.com/Megvii-BaseDetection/YOLOX.git 
!cd YOLOX && pip3 install -v -e
!git clone https://github.com/uclasystem/VQPy.git
!mv VQPy/vqpy ./
!pip install lap cython_bbox
!wget https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_x.pth

In [ ]:
import vqpy

In [ ]:
class Person(vqpy.VObjBase):
    pass

In [ ]:
class People_loitering_query(vqpy.QueryBase):
    @staticmethod
    def setting() -> vqpy.VObjConstraint:
        REGION = [
            (550, 550), (1162, 400), (1720, 720), (1430, 1072), (600, 1073)
            ]
        REGIONS = [REGION]

        filter_cons = {
            "__class__": lambda x: x == Person,
            "bottom_center": vqpy.utils.continuing(
                condition=vqpy.utils.within_regions(REGIONS),
                duration=10, name="in_roi"
            ),
        }
        select_cons = {
            "track_id": None,
            "coordinate": lambda x: str(x),  # convert to string for
                                             # JSON serialization
            # name in vqpy.continuing + '_periods' can be used in select_cons.
            "in_roi_periods": None,
        }
        return vqpy.VObjConstraint(
            filter_cons, select_cons, filename="loitering"
        )

In [ ]:
video_path = "./loitering.mp4"
save_folder = "./vqpy_outputs"
detector_model_dir = "./"

In [ ]:
vqpy.launch(
        cls_name=vqpy.COCO_CLASSES,
        cls_type={"person": Person},
        tasks=[People_loitering_query()],
        video_path=video_path,
        save_folder=save_folder,
        detector_name="yolox",
        detector_model_dir=detector_model_dir,
    )